## Lab: Cross-Validation and the Bootstrap
Reference: James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3

In [26]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                        summarize,
                        poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
                        (cross_validate,
                        KFold,
                        ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm


### The Validation Set Approach
Use the function `train_test_split()` to split the data into training train_test_
and validation sets. As there are 392 observations, we split into two equal
sets of size 196 using the argument test_size = `196`.

In [27]:
Auto = load_data('Auto')
Auto_train, Auto_test = train_test_split(Auto, test_size = 196, random_state = 0)

Now, we can fit the model by linear regression using observations from training set `Auto_train`.

In [28]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()
summarize(results)

,coef,std err,t,P>|t|
intercept,39.9055,1.009,39.537,0.0
horsepower,-0.1563,0.009,-17.333,0.0


We now use the `predict()` method of results evaluated on the model matrix for this model created using the validation data set. We also calculate the validation MSE of our model.

In [29]:
X_test = hp_mm.transform(Auto_test)
y_test = Auto_test['mpg']
test_pred = results.predict(X_test)
np.mean((y_test - test_pred)**2)

23.61661706966988

The estimate for the validation MSE of the linear regression fit is `23.62`.
Also estimation of the validation error for higher-degree polynomial regressions is:

In [30]:
def evalMSE(terms, response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    
    X_test = mm.transform(test)
    y_test = test[response]
    
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    
    return np.mean((y_test - test_pred)**2)
    

In [31]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_test)
MSE

array([23.61661707, 18.76303135, 18.79694163])

These error rates are `23.62`, `18.76`, and `18.80`, respectively. If we choose a
different training/validation split instead, then we can expect somewhat
different errors on the validation set.

In [32]:
Auto_train, Auto_test = train_test_split(Auto, test_size = 196, random_state = 3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_test)
MSE

array([20.75540796, 16.94510676, 16.97437833])

We find that the validation set error rates for the models with linear,quadratic, and cubic terms are `20.76`, `16.95`, and `16.97`, respectively.

A model that predicts mpg using a quadratic function of horsepower performs better than a model that involves only a linear function of horsepower,

### Cross-Validation

In [33]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                        X,
                        Y,
cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.231513517929212

The arguments to cross_validate() are as follows: an object with the appropriate `fit()`, `predict()`, and `score()` methods, an array of features  `X` and
a response `Y`. We also included an additional argument `cv` to `cross_validate()`;
specifying an integer K results in K-fold cross-validation. We have provided
a value corresponding to the total number of observations, which results in leave-one-out cross-validation (LOOCV). The `cross_validate()` function produces a dictionary with several components; we simply want the cross-validated test score here (MSE), which is estimated to be `24.23`.

We can repeat this procedure for increasingly complex polynomial fits.

In [34]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                        X,
                        Y,
    cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443031, 19.03320428])

There's a sharp drop in the estimated test MSE between the linear and quadratic fits, but then no clear improvement from using higher-degree polynomials.


In [35]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In the CV example above, we used `K = n`. Here we use `KFold()` to partition the data into `K = 10` random groups. We `KFold()` use random_state to set a random seed and initialize a vector `cv_error` in which we will store the CV errors corresponding to the polynomial fits of degrees one to five.

In [36]:
cv_error = np.zeros(5)
cv = KFold( n_splits=10,
            shuffle=True,
            random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(  M,
                            X,
                            Y,
                            cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848404, 19.13722016])

Notice that the computation time is much shorter than that of LOOCV.
(In principle, the computation time for LOOCV for a least squares linear
model should be faster than for K-fold CV)

The `cross_validate()` function is flexible and can take different splitting
mechanisms as an argument. For instance, one can use the `ShuffleSplit()` 
funtion to implement the validation set approach just as easily as K-fold
cross-validation.

In [37]:
validation = ShuffleSplit( n_splits=1,
                        test_size=196,
                        random_state=0)
results = cross_validate(  hp_model,
                        Auto.drop(['mpg'], axis=1),
                        Auto['mpg'],
                        cv=validation)
results['test_score']

array([23.61661707])

One can estimate the variability in the test error by running the following:

In [38]:
validation = ShuffleSplit(  n_splits=10,
                            test_size=196,
                            random_state=0)
results = cross_validate(   hp_model,
                            Auto.drop(['mpg'], axis=1),
                            Auto['mpg'],
                            cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.4218450941091847)

Note that this standard deviation is not a valid estimate of the sampling variability of the mean test score or the individual scores, since the
randomly-selected training samples overlap and hence introduce correlations. 

### The Bootstrap
No complicated mathematical calculations are required for Bootstrap method.
The `Portfolio` data set is used.The goal is to estimate the sampling variance of the parameter α. We will create a function alpha_func(), which takes as input a dataframe D assumed to have columns X and Y, as well as a vector idx indicating which
observations should be used to estimate α. The function then outputs the estimate for based on the selected observations.
#### Estimating the Accuracy of a Statistic of Interest

In [39]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
 cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
 return ((cov_[1,1]- cov_[0,1]) /
 (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

This function returns an estimate for based on applying the minimum variance formula  to the observations indexed by the argument `idx`. For
 instance, the following command estimates using all 100 observations.

In [40]:
alpha_func(Portfolio, range(100))

0.57583207459283

 Next we randomly select 100 observations from `range(100)`, with replacement. This is equivalent to constructing a new bootstrap data set and
 recomputing ˆbased on the new data set.

In [41]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace=True))

0.6074452469619004

This process can be generalized to create a simple function `boot_SE()` for
computing the bootstrap standard error for arbitrary functions that take
only a data frame as an argument.

In [42]:
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B- (first_ / B)**2)

Let’s use our function to evaluate the accuracy of our estimate of using B = 1,000 bootstrap replications.

In [43]:
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
alpha_SE

0.09118176521277699

The final output shows that the bootstrap estimate for SE(ˆα) is 0.0912.

####  Estimating the Accuracy of a Linear Regression Model



Here we use the bootstrap approach in order to assess the variability of the estimates for B0 and B1, the intercept and slope terms for the linear regres
sion model that uses horsepower to predict mpg in the Auto data set. 

To use our `boot_SE()` function, we must write a function (its first argu
ment) that takes a data frame `D` and indices `idx` as its only arguments. But
here we want to bootstrap a specific regression model, specified by a model
formula and data. 

We start by writing a generic function `boot_OLS()` for bootstrapping a
regression model that takes a formula to define the corresponding regres
sion. We use the `clone()` function to make a copy of the formula that can 
be refit to the new dataframe. This means that any derived features such
as those defined by` poly()` (which we will see shortly), will be re-fit on the
resampled data frame


In [73]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

This is not quite what is needed as the first argument to `boot_SE()`. The first
two arguments which specify the model will not change in the bootstrap
process, and we would like to freeze them. The function `partial()` from the 
`functools` module does precisely this: it takes a function as an argument,
and freezes some of its arguments, starting from the left. We use it to freeze
the first two model-formula arguments of `boot_OLS()`.

In [86]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')
rng = np.random.default_rng(0)

# Reset index to ensure compatibility
Auto = Auto.reset_index(drop=True)

results = np.array([
    hp_func(Auto, rng.choice(Auto.index, 392, replace=True)) for _ in range(10)
])
results

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

Next, we use the boot_SE() function to compute the standard errors of
 1,000 bootstrap estimates for the intercept and slope terms.

In [83]:
hp_se = boot_SE(hp_func,
                Auto,
                B=1000,
                seed=10)
hp_se

intercept     0.848807
horsepower    0.007352
dtype: float64

This indicates that the bootstrap estimate for $SE(\hat{\beta}_0)$ is 0.85, and that the bootstrap estimate for $SE(\hat{\beta}_1)$ is 0.0074.

Standard formulas can be used to compute the standard errors for the regression coefficients in a linear model. 


In [87]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

The standard error estimates for $\hat{\beta}_0$ and $\hat{\beta}_1$ are `0.717` for the intercept and `0.006` for the slope.  These are somewhat different from the estimates obtained using the bootstrap.

 Below we compute the bootstrap standard error estimates and the stan
dard linear regression estimates that result from fitting the quadratic model
 to the data.

In [88]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS,
                    quad_model,
                    'mpg')
boot_SE(quad_func, Auto, B=1000)


intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

We compare the results to the standard errors computed using `sm.OLS()`.

In [89]:
M = sm.OLS(Auto['mpg'],
quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64